In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


('1.13.1', 1, 'NVIDIA GeForce RTX 2070 with Max-Q Design')

In [ ]:
import torch
import torch_tensorrt
import timm
import time
import numpy as np
import torch.backends.cudnn as cudnn

torch.hub._validate_not_a_forked_repo=lambda a,b,c: True

#efficientnet_b0 = timm.create_model('efficientnet_b0',pretrained=True)
#efficientnet = timm.create_model('efficientnet_b4',pretrained=True)

In [3]:
m = EffNet().to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

18297.787

In [5]:
stem = "1.29_avg"

m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models_deploy/m{stem}.torch"), strict=False)
backbone = m.backbone

In [8]:
# https://developer.nvidia.com/blog/accelerating-inference-up-to-6x-faster-in-pytorch-with-torch-tensorrt/
import torch.backends.cudnn as cudnn
cudnn.benchmark = True

def benchmark(model, input_shape=(1024, 3, 512, 512), dtype='fp32', nwarmup=50, nruns=1000):
    input_data = torch.randn(input_shape)
    input_data = input_data.to("cuda")
    if dtype=='fp16':
        input_data = input_data.half()
        
    print("Warm up ...")
    with torch.no_grad():
        for _ in range(nwarmup):
            features = model(input_data)
    torch.cuda.synchronize()
    print("Start timing ...")
    timings = []
    with torch.no_grad():
        for i in range(1, nruns+1):
            start_time = time.time()
            pred_loc  = model(input_data)
            torch.cuda.synchronize()
            end_time = time.time()
            timings.append(end_time - start_time)
            if i%10==0:
                print('Iteration %d/%d, avg batch time %.2f ms'%(i, nruns, np.mean(timings)*1000))

    print("Input shape:", input_data.size())
    print('Average throughput: %.2f images/second'%(input_shape[0]/np.mean(timings)))

In [10]:
N_CHANNELS_MODEL

4

In [11]:
input_shape = (1, N_CHANNELS_MODEL, IMG_HEIGHT, IMG_WIDTH)

In [12]:
model = backbone.eval().to("cuda")
benchmark(model, input_shape=input_shape, nruns=10)

Warm up ...
Start timing ...
Iteration 10/10, avg batch time 30.71 ms
Input shape: torch.Size([1, 4, 360, 1440])
Average throughput: 32.57 images/second


In [13]:
traced_model = torch.jit.trace(model, torch.randn(input_shape).to("cuda"))
benchmark(traced_model, input_shape=input_shape, nruns=10)

Warm up ...
Start timing ...
Iteration 10/10, avg batch time 25.33 ms
Input shape: torch.Size([1, 4, 360, 1440])
Average throughput: 39.48 images/second


In [15]:
import torch_tensorrt

In [16]:
%%time
trt_model = torch_tensorrt.compile(model, 
    inputs= [torch_tensorrt.Input(input_shape)],
    enabled_precisions= { torch_tensorrt.dtype.half} # Run with FP16
)

ERROR: [Torch-TensorRT TorchScript Conversion Context] - 2: [virtualMemoryBuffer.cpp::resizePhysical::145] Error Code 2: OutOfMemory (no further information)
ERROR: [Torch-TensorRT TorchScript Conversion Context] - 2: [virtualMemoryBuffer.cpp::resizePhysical::145] Error Code 2: OutOfMemory (no further information)
Try decreasing the workspace size with IBuilderConfig::setMemoryPoolLimit().
ERROR: [Torch-TensorRT TorchScript Conversion Context] - 2: [virtualMemoryBuffer.cpp::resizePhysical::145] Error Code 2: OutOfMemory (no further information)
ERROR: [Torch-TensorRT TorchScript Conversion Context] - 2: [virtualMemoryBuffer.cpp::resizePhysical::145] Error Code 2: OutOfMemory (no further information)
Try decreasing the workspace size with IBuilderConfig::setMemoryPoolLimit().
ERROR: [Torch-TensorRT TorchScript Conversion Context] - 2: [virtualMemoryBuffer.cpp::resizePhysical::145] Error Code 2: OutOfMemory (no further information)
ERROR: [Torch-TensorRT TorchScript Conversion Context] -

In [17]:
trt_model

RecursiveScriptModule(original_name=EfficientNet_trt)

In [18]:
torch.jit.save(trt_model, f"{BESPOKE_ROOT}/trt_models/backbone_trt.jit.pt")

In [19]:
benchmark(trt_model, input_shape=input_shape, nruns=10)

Warm up ...
Start timing ...
Iteration 10/10, avg batch time 9.67 ms
Input shape: torch.Size([1, 4, 360, 1440])
Average throughput: 103.41 images/second


In [6]:
import torch.nn as nn


In [8]:
l = nn.Linear(3,5)

[torch.float32, torch.float32]